Obtain

In [ ]:
%matplotlib inline

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
plt.style.use('seaborn')
df = pd.read_csv('kc_house_data.csv')
df.head()

Scrub

In [ ]:
df.info() # check datatypes

In [ ]:
df = df.drop(columns=['id'])

In [ ]:
df.isna().sum()

In [ ]:
df = df.dropna(subset=['view'])

Explore

Model

Interpret